# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets


# Import API key
from config import g_key

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [8]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df

,City_ID,City_ID.1,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
...,...,...,...,...,...,...,...,...,...
347,347,695,Kostomuksha,RU,79.27,overcast clouds,64.57,30.58,Apartments on Lenina 17
348,348,700,Sabratah,LY,85.15,clear sky,32.79,12.49,مقبرة المجاهديه
349,349,704,Wuwei,CN,75.00,overcast clouds,37.93,102.64,Thank Inn Chain Hotel gansu wuwei liangzhou di...
350,350,705,Chake Chake,TZ,78.87,few clouds,-5.25,39.77,Hotel Archipelago


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [6]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [7]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [30]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df["City"]=='Northam']
vacation_end =  vacation_df.loc[vacation_df["City"]=='Barrhead']
vacation_stop1 =  vacation_df.loc[vacation_df["City"]=='Margate']
vacation_stop2 =  vacation_df.loc[vacation_df["City"]=='Liverpool']
vacation_stop3 =  vacation_df.loc[vacation_df["City"]=='Warrington']

In [31]:
italy_df = vacation_df.query("Country == 'GB'")
italy_df

,City_ID,City_ID.1,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
81,81,200,Barrhead,GB,80.01,scattered clouds,55.80,-4.39,The Gleniffer Hotel
99,99,237,Margate,GB,84.99,clear sky,51.38,1.39,Premier Inn Margate hotel
207,207,438,Liverpool,GB,88.00,few clouds,53.41,-2.98,Premier Inn Liverpool City Centre (Moorfields)...
304,304,607,Scarborough,GB,73.99,clear sky,54.28,-0.40,Ambassador
317,317,633,Warrington,GB,88.00,clear sky,53.39,-2.58,Villaggio Hotel


## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [32]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(map(tuple,(vacation_start[['Lat', 'Lng']].to_numpy())))[0]
end = tuple(map(tuple,(vacation_end[['Lat', 'Lng']].to_numpy())))[0]
stop1 = tuple(map(tuple,(vacation_stop1[['Lat', 'Lng']].to_numpy())))[0]
stop2 = tuple(map(tuple,(vacation_stop2[['Lat', 'Lng']].to_numpy())))[0]
stop3 = tuple(map(tuple,(vacation_stop3[['Lat', 'Lng']].to_numpy())))[0]

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [33]:
# Define a new figure object

fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3,], travel_mode='DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [34]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City_ID.1,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
1,99,237,Margate,GB,84.99,clear sky,51.38,1.39,Premier Inn Margate hotel
2,207,438,Liverpool,GB,88.00,few clouds,53.41,-2.98,Premier Inn Liverpool City Centre (Moorfields)...
3,317,633,Warrington,GB,88.00,clear sky,53.39,-2.58,Villaggio Hotel


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [41]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [50]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Define a new figure object
fig = gmaps.figure(center=(50.0, 50.0), zoom_level=3.5)

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))